<a href="https://colab.research.google.com/github/JohanaR10/ML/blob/main/mml_clase1_pytorch_mpg_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mastering Machine Learning 2025

### Clase 1: Introducción a redes neuronales en pytorch

- Redes densas
- Redes para regresión
- Funciones de activación y pérdida


Empecemos importando numpy y pandas

In [ ]:
import numpy as np
import pandas as pd

Montemos el drive de google para leer archivos almacenados allí

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Carguemos el archivo auto-mpg.data (disponible en Bloque Neón) usando pandas. Note que debemos incluir los nombres de las columnas, además de indicar los caracteres para separación, nas y comentarios.

In [ ]:
df = pd.read_csv('gdrive/My Drive/25_mml/sesiones/auto-mpg.data',
                 names=['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin'],
                 sep=' ',
                 na_values='?',
                 comment='\t',
                 skipinitialspace=True
                 )

Exploremos las primeras filas del dataframe.

In [ ]:
df.head()

- MPG: miles per gallon
- Cylinders: # de cilindros
- Displacement: volumen desplazado por los pistones
- Horsepower: caballos de potencia
- Weight: peso
- Acceleration: aceleración
- Model Year: año/modelo
- Origin: 1:USA, 2:Europa, 3:Japón

Exploremos el tamaño del df

In [ ]:
df.shape

Identificamos NAs en los datos

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df

Separamos los datos en entrenamiento y prueba

In [ ]:
from sklearn import model_selection

In [ ]:
train, test = model_selection.train_test_split(df, train_size=0.8, random_state=100)
train.head()

In [ ]:
train.shape

In [ ]:
test = df.drop(train.index)
test.head()

In [ ]:
print(train.shape)
print(test.shape)

Realicemos una exploración descriptiva de los datos, calculando inicialmente estadísticas de cada variable continua

In [ ]:
train.describe()

Exploremos relaciones bivariadas con pairplot de seaborn

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(train)

Calculemos la matriz de correlacion

In [ ]:
train.corr()

In [ ]:
sns.heatmap(train.corr())

In [ ]:
numeric_cols = ['Cylinders', 'Displacement','Horsepower','Weight','Acceleration', 'Model Year']

In [ ]:
train_norm , test_norm = train.copy(), test.copy()

for col in numeric_cols:
  mean = train[col].mean()
  std = train[col].std()

  train_norm[col] = ( train_norm[col] - mean) /std
  test_norm[col] = (test_norm[col] - mean) /std

In [ ]:
train_norm.head()

Codificación one-hot para Origin y generación de datos de entrada x_train y x_test, en formato tensor

In [ ]:
import torch
from torch.nn.functional import one_hot

# determinar número de categorías
n_categories = len(set(train_norm['Origin']))

# codificación one hot para Origin - train set
origin_encoded = one_hot(torch.from_numpy(train_norm['Origin'].values % n_categories))
x_train_numeric = torch.tensor(train_norm[numeric_cols].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

# codificación one hot para Origin - test set
origin_encoded = one_hot(torch.from_numpy(test_norm['Origin'].values % n_categories))
x_test_numeric = torch.tensor(test_norm[numeric_cols].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

Datos de salida en formato tensor

In [ ]:
y_train = torch.tensor(train_norm['MPG'].values).float()
y_test = torch.tensor(test_norm['MPG'].values).float()

Crear un Dataset y un DataLoader para el entrenamiento de los modelos

In [ ]:
from torch.utils.data import Dataset, TensorDataset, DataLoader

train_ds = TensorDataset(x_train, y_train)
batch_size = 16
torch.manual_seed(42)

train_dl = DataLoader(train_ds, batch_size, shuffle = True)

Crear un primer modelo con dos capas ocultas de 8 y 4 neuronas (función de activación ReLU) y una salida lineal (regresión)

In [ ]:
import torch.nn as nn

hidden_units = [8, 4]
input_size = x_train.shape[1]

all_layers = []
for hidden_units_layer in hidden_units:
  layer = nn.Linear(input_size, hidden_units_layer)
  all_layers.append(layer)
  all_layers.append(nn.ReLU())
  input_size = hidden_units_layer

all_layers.append(nn.Linear(hidden_units[-1], 1))
model = nn.Sequential(*all_layers)

model

Entrenar el modelo usando descenso del gradiente estocástico y función de pérdida MSE (mean squared error)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

torch.manual_seed(1)
num_epochs = 200
log_epochs = 20

for epoch in range(num_epochs):
  loss_hist_train = 0
  for x_batch, y_batch in train_dl:
    pred = model(x_batch)[:,0]
    loss = loss_fn(pred, y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    loss_hist_train += loss.item()

  if epoch % log_epochs == 0:
    print(f'Epoch {epoch} Loss {loss_hist_train/len(train_dl):.4f}')

Predicción de un nuevo dato de prueba y evaluación con MSE y MAE (mean absolute error)

In [ ]:
with torch.no_grad():
  pred = model(x_test.float())[:,0]
  loss = loss_fn(pred, y_test)
  print(f'Test MSE: {loss.item():.4f}')
  print(f'Test MAE: {nn.L1Loss()(pred,y_test).item():.4f}')